In [103]:
from IPython.display import Image
from time import sleep
from sklearn.metrics import r2_score
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


"""Onely have 1 DATASET_NAME uncommented."""
# DATASET_NAME = 'kiosk_weather_by_day.csv'
#DATASET_NAME = 'cycle_weather_by_day.csv'
DATASET_NAME = 'cycle_weather_by_day_duration_avg.csv'

ts = '{:%Y%m%d%H%M%S}'.format(datetime.now())

In [104]:

from amb_sdk.sdk import DarwinSdk
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

status, msg = ds.auth_login_user('danny.diaz@utexas.edu', 'JcXCnQ7Wx2')
if not status:
    print(msg)
else:
    print('You are logged in: ',status)


You are logged in:  True


In [105]:
# ds.get_info()
# ds.lookup_job_status()
# ds.lookup_job_status_name(job_name)
# ds.delete_job(job_name)
# ds.stop_job(job_name) 


# Data Upload
**Read dataset and view a file snippet**

In [106]:
df = pd.read_csv('./data/'+DATASET_NAME)
#print(df.dtypes)
df.head()

,Date,Month,Year,Weekday,Avg Trip Duration,Total Rides,TempHighF,TempAvgF,DewPointHighF,DewPointAvgF,WindHighMPH,WindAvgMPH,PrecipitationSumInches,Events,PrecipBins
0,2013-12-21,12,2013,5,29.136,103,74,60,67,49,20,4,0.460,"Rain , Thunderstorm",0.1-0.5
1,2013-12-22,12,2013,6,40.462,117,56,48,43,36,16,6,0.000,Sunny,0
2,2013-12-23,12,2013,0,55.604,96,58,45,31,27,8,3,0.000,Sunny,0
3,2013-12-24,12,2013,1,31.965,85,61,46,36,28,12,4,0.000,Sunny,0
4,2013-12-25,12,2013,2,49.297,145,58,50,44,40,10,2,0.005,Sunny,0.1-0.5


**Upload dataset to Darwin**

In [107]:
status, dataset = ds.upload_dataset('./data/'+DATASET_NAME)
print(status)
print(dataset)

if not status:
    print(dataset)

False
400: BAD REQUEST - {"message": "Dataset already exists"}

400: BAD REQUEST - {"message": "Dataset already exists"}



   # Clean Data

In [108]:
"""
The avg trip duration has a few outliers that are throwing off the data. Need to remove any trips over 
60 minutes.
"""
#target = 'Avg Trip Duration'
target = 'Total Rides'
status, job_id = ds.clean_data(dataset_name=DATASET_NAME, 
                               target= target)
if not status:
    print(job_id)
else:
    ds.wait_for_job(job_id['job_name'])
    print('\nData has been successfully cleaned!')

{'status': 'Requested', 'starttime': '2019-04-21T10:42:47.90888', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['cycle_weather_by_day_duration_avg.csv'], 'artifact_names': ['baeffe1613474188b96c320873218403'], 'model_name': None, 'job_error': None}
{'status': 'Failed', 'starttime': '2019-04-21T10:42:47.90888', 'endtime': '2019-04-21T10:42:48.576781', 'percent_complete': 100, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['cycle_weather_by_day_duration_avg.csv'], 'artifact_names': ['baeffe1613474188b96c320873218403'], 'model_name': None, 'job_error': "FailedDownload: Failed to download entity of type File from location ['datasets/20ec826a-4fc6-11e9-b56b-7f7f89ca7302_cycle_weather_by_day_duration_avg.csv.csv']"}

Data has been successfully cleaned!


# Create and Train Model

We will now build a model that will learn the class labels in the target column.<br> In the default boston dataset, the target column is "Assessed_Value". <br> You will have to specify your own target name for your custom dataset. <br> You can also increase max_train_time for longer training.




In [109]:
model = target + "_model0" + ts
status, job_id = ds.create_model(dataset_names = DATASET_NAME, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Taken', 'starttime': '2019-04-21T10:43:04.365298', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['cycle_weather_by_day_duration_avg.csv'], 'artifact_names': None, 'model_name': 'Total Rides_model020190421104244', 'job_error': None}
{'status': 'Failed', 'starttime': '2019-04-21T10:43:04.365298', 'endtime': '2019-04-21T10:43:04.550275', 'percent_complete': 100, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['cycle_weather_by_day_duration_avg.csv'], 'artifact_names': None, 'model_name': 'Total Rides_model020190421104244', 'job_error': 'FailedDownload: Failed to download entity of type Object from location artifacts/20ec826a-4fc6-11e9-b56b-7f7f89ca7302_baeffe1613474188b96c320873218403/data_profiler_main'}


# Analyze Model
Analyze model provides feature importance ranked by the model. <br> It indicates a general view of which features pose a bigger impact on the model

In [110]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

403: FORBIDDEN - {"message": "Analyze model is not supported for models with an unknown type"}



TypeError: string indices must be integers

Show the 10 most important features of the model.

In [ ]:
feature_importance[:10]

# Predictions
**Perform model prediction on the the training dataset.**

In [ ]:
status, artifact = ds.run_model(DATASET_NAME, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

Download predictions from Darwin's server.

In [ ]:
status, prediction = ds.download_artifact(artifact['artifact_name'])

Create plots comparing predictions with actual target

In [ ]:
#Plot predictions vs actual
plt.plot(df[target], prediction[target], '.')
plt.plot([0,200],[0,200],'--k')
print('R^2 : ', r2_score(df[target], prediction[target]))

In [ ]:
status, model_type = ds.lookup_model_name(model)
print(model_type['description']['best_genome'])

In [ ]:
ds.delete_model(model)

In [ ]:
#ds.lookup_job_status_name(analyze_id['job_name'])